About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import nltk

In [ ]:
# Download the 'stopwords' dataset, which contains a list of common stop words
# Stop words are words like 'the', 'and', 'is', etc., that are often filtered out in text processing tasks as they carry less meaningful information.
nltk.download('stopwords')

# Download the 'wordnet' lexical database, which is useful for finding synonyms, defining words, and other linguistic tasks. 
# WordNet can help improve text processing by providing a semantic understanding of words and their relationships.
nltk.download('wordnet')

Data Pre-processing

In [3]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv(r'dataset\train.csv' , encoding='unicode_escape')

In [4]:
news_dataset.shape

(20800, 5)

In [5]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
# Replace NaN values with empty strings
news_dataset = news_dataset.fillna('')

# Concatenate author, title, and text into 'content'
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title'] + ' ' + news_dataset['text']

In [8]:
# Function for cleaning text
def clean_text(content):
    content = re.sub(r'[^\x00-\x7F]+', '', content) # Remove non-ASCII characters
    content = re.sub(r'\s+', ' ', content).strip()  # Replace multiple whitespace with a single space and trim leading/trailing spaces.
    return content # Return cleaned text

In [9]:
news_dataset['content'] = news_dataset['content'].apply(clean_text) # Apply the clean_text function to each entry in the 'content' column.

Lemmatization:

Lemmatization is the process of reducing a word to its base or dictionary form, considering its context and part of speech.

Example: better, best → good

In [10]:
# Function for lemmatization
lemmatizer = WordNetLemmatizer() # Initialize the lemmatizer
stop_words = set(stopwords.words('english')) # Create a set of English stopwords
def lemmatize(content): 
    content = re.sub('[^a-zA-Z]', ' ', content) # Remove any characters that are not letters and replace them with a space
    content = content.lower() # Convert the content to lowercase
    content = content.split() # Split the content into individual words
    content = [lemmatizer.lemmatize(word) for word in content if word not in stop_words]  # Lemmatize each word and filter out stopwords
    return ' '.join(content)  # Join the lemmatized words back into a single string and return

In [11]:
news_dataset['content'] = news_dataset['content'].apply(lemmatize) # Apply the lemmatization function to the 'content' column of the news_dataset 

In [12]:
print(news_dataset['content'])

0        darrell lucus house dem aide didnt even see co...
1        daniel j flynn flynn hillary clinton big woman...
2        consortiumnews com truth might get fired truth...
3        jessica purkiss civilian killed single u airst...
4        howard portnoy iranian woman jailed fictional ...
                               ...                        
20795    jerome hudson rapper trump poster child white ...
20796    benjamin hoffman n f l playoff schedule matchu...
20797    michael j de la merced rachel abrams macys sai...
20798    alex ansary nato russia hold parallel exercise...
20799    david swanson keep f alive david swanson autho...
Name: content, Length: 20800, dtype: object


In [13]:
# Step 4: Separating the Data and Labels
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [14]:
print(X)
print(Y)

['darrell lucus house dem aide didnt even see comeys letter jason chaffetz tweeted house dem aide didnt even see comeys letter jason chaffetz tweeted darrell lucus october subscribe jason chaffetz stump american fork utah image courtesy michael jolley available creative common license apology keith olbermann doubt worst person world weekfbi director james comey according house democratic aide look like also know second worst person well turn comey sent infamous letter announcing fbi looking email may related hillary clinton email server ranking democrat relevant committee didnt hear comey found via tweet one republican committee chairman know comey notified republican chairman democratic ranking member house intelligence judiciary oversight committee agency reviewing email recently discovered order see contained classified information long letter went oversight committee chairman jason chaffetz set political world ablaze tweet fbi dir informed fbi learned existence email appear pertine

In [15]:
print(news_dataset['content'])

0        darrell lucus house dem aide didnt even see co...
1        daniel j flynn flynn hillary clinton big woman...
2        consortiumnews com truth might get fired truth...
3        jessica purkiss civilian killed single u airst...
4        howard portnoy iranian woman jailed fictional ...
                               ...                        
20795    jerome hudson rapper trump poster child white ...
20796    benjamin hoffman n f l playoff schedule matchu...
20797    michael j de la merced rachel abrams macys sai...
20798    alex ansary nato russia hold parallel exercise...
20799    david swanson keep f alive david swanson autho...
Name: content, Length: 20800, dtype: object


In [16]:
Y.shape

(20800,)

CONVERTING THE TEXTUAL DATA TO NUMERICAL DATA/FEATURE VECTOR

1)tf (term freequency):-which counts the number of time a perticular words is repearting in a documents.
2)idf (inverse document freequency):-Measure the word is a rare word or common word in a document

In [17]:
#Converting Textual Data to Numerical Data
vectorizer = TfidfVectorizer(max_df=0.7, min_df=5, stop_words='english') 
X = vectorizer.fit_transform(X)

In [18]:
print(X)

  (0, 8518)	0.2614664537670539
  (0, 21031)	0.15024953295253837
  (0, 16788)	0.09862915172747089
  (0, 9015)	0.1062301855032697
  (0, 775)	0.14020015141322165
  (0, 9587)	0.12411822877467002
  (0, 6628)	0.12385470266716889
  (0, 20391)	0.174301874815182
  (0, 18840)	0.16965956677877497
  (0, 5540)	0.6011837103550017
  (0, 36782)	0.08594403117510611
  (0, 24637)	0.035222170020158115
  (0, 34428)	0.03035152048786258
  (0, 34340)	0.038999296478856915
  (0, 1195)	0.011912482627209809
  (0, 13752)	0.04106812519488518
  (0, 37804)	0.06604583670568329
  (0, 17262)	0.020406112698087
  (0, 7758)	0.06373993714233822
  (0, 22416)	0.019499884864808043
  (0, 2428)	0.021321107446251528
  (0, 7899)	0.02817810566552832
  (0, 6706)	0.020228081997562344
  (0, 20499)	0.02792710633227959
  (0, 1628)	0.03189278749130583
  :	:
  (20799, 38487)	0.08249704443395182
  (20799, 20992)	0.20395706504484448
  (20799, 26098)	0.044889930613211455
  (20799, 2022)	0.04726440817112628
  (20799, 6167)	0.0347358782489955


Splitting the dataset to training & test data

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Random Forest 

In [20]:
# n_estimators: Specifies the number of trees in the forest
# A higher number can improve accuracy but may increase training time
# n_estimators=100 Using 100 decision trees in the ensemble
# random_state=2 Sets a seed for reproducibility of results
model = RandomForestClassifier(n_estimators=100, random_state=2)

In [21]:
model.fit(X_train, Y_train)

RandomForestClassifier(random_state=2)

Evaluation

accuracy score

In [22]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [23]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  1.0


In [24]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [25]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9490384615384615


In [26]:
#Save the model and vectorizer
joblib.dump(model, 'random_forest_model.sav')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

Making a Predictive System

In [27]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [28]:
print(Y_test[3])

0
